# Case Study - Homeless

In [9]:
import pandas as pd
from config import *

df = pd.read_csv('source/homeless.csv')
df.head()

,ITTER107,Territory,Sex,Age,Citizenship,Value
0,ITC1,Piemonte,M,TOTAL,ITL,4218
1,ITC1,Piemonte,F,TOTAL,ITL,1496
2,ITC1,Piemonte,T,TOTAL,ITL,5714
3,ITC2,Valle d'Aosta,M,TOTAL,ITL,41
4,ITC2,Valle d'Aosta,F,TOTAL,ITL,17


Select only some fields

In [10]:
df.Citizenship.unique()

array(['ITL', 'FRGAPO', 'TOTAL'], dtype=object)

In [11]:
df['Territory'] = df['Territory'].str.replace('Trentino Alto Adige', 'Trentino-Alto Adige/Südtirol')

Focus on total age, total sex and total citizenship

In [12]:
df_tot = df[(df['Age'] == 'TOTAL') & (df['Sex'] == 'T') &  (df['Citizenship'] == 'TOTAL')]

## Draw a preliminary map

In [13]:
import altair as alt

# Load the TopoJSON file by URL
url = "https://raw.githubusercontent.com/openpolis/geojson-italy/master/topojson/limits_IT_regions.topo.json"


source = alt.topo_feature(url, "regions")

chart = alt.Chart(source).mark_geoshape().encode(
    tooltip='properties.reg_name:N',
    color=alt.Color('Value:Q')
).project('mercator').properties(
    width=500,
    height=500
).transform_lookup(
    lookup='properties.reg_name',
    from_=alt.LookupData(df_tot, 'Territory', ['Territory', 'Value'])
).properties(title='Homeless in Italy in 2021')

configure_layout(chart)

alt.Chart(...)

# Focus on the key message

In [14]:
df_tot['Territory'] = df_tot['Territory'].str.replace('Trentino-Alto Adige/Südtirol','Trentino Alto Adige')

chart = alt.Chart(df_tot).mark_bar(color=color).encode(
    y = alt.Y('Territory:N', sort='-x', axis=alt.Axis(title='')),
    x = alt.X('Value:Q', axis=alt.Axis(title='Number of homeless')),
   
).properties(title='Homeless in Italy in 2021')

configure_layout(chart)

/var/folders/vc/8c80t7jd1dq_mmp6cwgwxt640000gn/T/ipykernel_1634/589044412.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tot['Territory'] = df_tot['Territory'].str.replace('Trentino-Alto Adige/Südtirol','Trentino Alto Adige')


alt.Chart(...)

In [15]:
chart

alt.Chart(...)

In [16]:
chart.save('chart.html')

Normalize by population

In [17]:
population = pd.read_csv('source/population.csv')
population.head()

,ITTER107,Territory,Sex,Age,Value
0,ITF1,Abruzzo,M,Y50-54,50764
1,ITF1,Abruzzo,T,Y55-59,102824
2,ITF1,Abruzzo,T,Y80-84,50370
3,ITF1,Abruzzo,M,Y25-29,33107
4,ITF1,Abruzzo,M,Y45-49,48230


In [18]:
pop_tot = population[(population['Age'] == 'TOTAL') & (population['Sex'] == 'T')]

In [19]:
len(df_tot), len(pop_tot)

(20, 20)

In [20]:
df_tot.columns

Index(['ITTER107', 'Territory', 'Sex', 'Age', 'Citizenship', 'Value'], dtype='object')

In [21]:
df_tot.Territory.unique()

array(['Piemonte', "Valle d'Aosta", 'Liguria', 'Lombardia',
       'Trentino Alto Adige', 'Veneto', 'Friuli-Venezia Giulia',
       'Emilia-Romagna', 'Toscana', 'Umbria', 'Marche', 'Lazio',
       'Abruzzo', 'Molise', 'Campania', 'Puglia', 'Basilicata',
       'Calabria', 'Sicilia', 'Sardegna'], dtype=object)

In [22]:
pop_tot.Territory.unique()

array(['Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Emilia-Romagna',
       'Friuli-Venezia Giulia', 'Lazio', 'Liguria', 'Lombardia', 'Marche',
       'Molise', 'Piemonte', 'Puglia', 'Sardegna', 'Sicilia', 'Toscana',
       'Trentino Alto Adige', 'Umbria', "Valle d'Aosta", 'Veneto'],
      dtype=object)

Join the two datasets

In [23]:
df_tot = df_tot[['Value', 'ITTER107']]
pop_tot = pop_tot[['Value', 'ITTER107','Territory']]

In [24]:
df3 = pop_tot.set_index('ITTER107').join(df_tot.set_index('ITTER107'),lsuffix='_pop', rsuffix='_hom').reset_index()

Calculate ratio: number of homeless people over 1000 inhabitants

In [25]:
df3['Ratio'] = df3['Value_hom']/df3['Value_pop']*1000

In [26]:
chart = alt.Chart(df3).mark_bar().encode(
    y = alt.Y('Territory', sort='-x', axis=alt.Axis(title='')),
    x = alt.X('Ratio', axis=alt.Axis(tickCount=4,title='')),
    color=alt.condition(alt.datum.Ratio > 2, alt.value(iColor), alt.value('lightgray'))
   
).properties(width=500,title='Number of homeless people in a population of 1,000')

configure_layout(chart)

alt.Chart(...)

# Adding a context

In [27]:
chart = alt.Chart(df3).mark_bar().encode(
    y = alt.Y('Territory', sort='-x', axis=alt.Axis(title='')),
    x = alt.X('Ratio', axis=alt.Axis(tickCount=4,title='')),
    color=alt.condition(alt.datum.Ratio > 2, alt.value(iColor), alt.value('lightgray'))
   
).properties(width=500,title=alt.TitleParams(
    text=["Together, Let's Make a Difference:","Support Our Project to Help the Homeless!"],
    subtitle=['Homelessness is a heartbreaking reality that leaves individuals and families without a stable home,','leading to devastating consequences such as poor health and social isolation.'],
    subtitleFontSize=18
))


configure_layout(chart)

alt.Chart(...)

In [28]:

# Create an Altair chart with an image mark
image1 = alt.Chart(pd.DataFrame({'image_url': ['source/homeless1.png']})).mark_image(
    width=200,
    height=200,
).encode(
    url='image_url',
    x=alt.value(0),  # pixels from left
    y=alt.value(50)
)

image2 = alt.Chart(pd.DataFrame({'image_url': ['source/homeless2.png']})).mark_image(
    width=200,
    height=200,
).encode(
    url='image_url',
    x=alt.value(0),  # pixels from left
    y=alt.value(300)
)

configure_layout(image1 + image2| chart)

alt.HConcatChart(...)

# Adding next steps

In [29]:
ns = pd.read_csv('source/next_steps.csv')
ns.head()

,Category,Allocation
0,Shelter and Housing,35
1,Job Training,25
2,Education,20
3,Healthcare,10
4,Basic Necessities,10


In [32]:
donuts = None
for index, row in ns.iterrows():
    # Create a pie chart for the current row
    curr_ns = pd.DataFrame(
        {'Category': ['A', 'B'],
         'Value': [row['Allocation'], 100-row['Allocation']]
        }
    )
    donut = alt.Chart(curr_ns).mark_arc(outerRadius=30,innerRadius=20).encode(
        theta=alt.Theta("Value:Q",stack=True),
        color=alt.Color("Category:N",scale=alt.Scale(range=[iColor, 'lightgray']),legend=None)
    )
    title = alt.Chart(curr_ns).mark_text(text=row['Category'], y=0, size=16)
    text = alt.Chart(curr_ns).mark_text(text=f"{row['Allocation']}%", color=iColor, size=16)
    donut = donut.properties(
        height=100, 
        width=100
    )
    if index == 0:
        donuts = title+donut+text
    else:
        donuts = alt.hconcat(donuts, title+donut+text)
donuts = donuts.properties(title='Our visionary plan to harness the funds')
configure_layout(donuts)

alt.HConcatChart(...)

In [31]:
configure_layout(alt.vconcat(image1 + image2| chart,donuts))

alt.VConcatChart(...)